In [1]:
import openpyxl
import os

In [2]:
class File:

    def __init__(self, file_name):
        self.file_name = file_name
    

    # открыть файл для чтения
    def open_file(self):
        file = openpyxl.load_workbook(f'{self.file_name}')
        sheet = file.active
        return sheet


    # параметры файла
    def max_row(self, sheet):
        return sheet.max_row
    def max_col(self, sheet):
        return sheet.max_column

    # определение координат начала и конца нужных данных 
    # (предполагается, что структура файлов та же, что в примере)
    def mincell(self):
        return 'E2' # по умолчанию
    def maxcell(self, sheet, max_row, max_col):
        return sheet.cell(row = max_row, column=max_col).coordinate

    
    # чтение файла построчно
    def read_file_by_line(self, sheet, mincell, maxcell):
        lst_file_by_line = list()
        for rows in sheet[mincell:maxcell]:
            lst_in = list()
            for cell in rows:
                # значение ячейки
                cell_val = cell.value

                # замена None на 0
                if cell_val == None:
                    cell_val = 0
                
                lst_in.append(cell_val)
                
            lst_file_by_line.append(lst_in)
        
        return lst_file_by_line


    # значения ячеек по столбцам
    def column_values(self, lst_file_by_line):
        lst_column_values = list()
        for i in range(len(lst_file_by_line[0])):
            lst_column_in = list()
            for j in range(len(lst_file_by_line)):
                lst_column_in.append(lst_file_by_line[j][i])
            lst_column_values.append(lst_column_in)
        
        return lst_column_values
    

    # объединение значений по два столбца
    def values_to_couple_PlanFact(self, lst_column_values):
        lst_couple = list()
        for i in range(0,len(lst_column_values),2):
            lst_couple.append(lst_column_values[i:i+2])
        
        return lst_couple


    # получение списка успешностей сотрудников
    def difference(self, lst_couple):

        # success_workers_lst - список, содержащий успешность 
        # каждого сотрудника в порядке их записи в файле
        success_workers_lst = list()

        # элементы списка difference_lst - разница между фактом
        # и планом для каждого проекта для одного из сотрудников
        difference_lst = list()

        for i in range(len(lst_couple)):
            for j in range(len(lst_couple[0][0])):
                difference_lst.append(float(lst_couple[i][1][j])-float(lst_couple[i][0][j]))

            # присоединение разности к списку
            success_workers_lst.append(sum(difference_lst))

            # очистка списка перед тем, как приступить к обработке
            # успешности следующего сотрудника
            difference_lst.clear()
        
        return success_workers_lst

    # данные в ячейках, относящиеся к одному сотруднику, должны
    # совпадать и быть корректными
    def isperson(self,person1, person2):
        # проверка для каждой ячейки, что ФИО в нужном формате
        for person in [person1, person2]:
            person_FIO = person.split()

            # должно быть два элемента - фамилия и И.О.
            # также проверяется, не содержит ли фамилия лишних символов
            if len(person_FIO) != 2 or not person_FIO[0].isalpha():
                return False
                    
            # исследуются И.О.
            person_IO = person_FIO[1].split('.')[:-1]
            if len(person_IO) == 2:
                for elem in person_IO:
                    if not elem.isalpha():
                        return False
            else:
                return False
        
        # совпадают ли данные ячеек
        if person1 != person2:
            return False
            
        return True


    # получение имён сотрудников
    def name_workers(self, sheet, max_col):
        workers = list()
        for col in range(5,max_col+1,2):
            person1 = (sheet.cell(row=1, column=col).value)[:-5]
            person2 = (sheet.cell(row=1, column=col+1).value)[:-5]
            if self.isperson(person1, person2):
                workers.append(person1)
            else:
                print(f'Есть некорректные данные в ячейке {1,col} или {1,col+1}, '
                f'сотрудник {person1}/{person2} не учтён!')
        return workers


    # успешность каждого сотрудника поимённо
    def success_dict(self, workers, success_workers_lst):
        success_workers_dict = dict()
        for i in range(len(workers)):
            success_workers_dict[workers[i]] = success_workers_lst[i]
        
        return success_workers_dict
    



# обработка одного файла
def success_by_one_file(file_name):
    
    file = File(file_name)

    sheet = file.open_file()
    max_row, max_col = file.max_row(sheet), file.max_col(sheet)
    mincell =  file.mincell()
    maxcell = file.maxcell(sheet,max_row,max_col)

    
    lst_file_by_line = file.read_file_by_line(sheet, mincell, maxcell)
    lst_column_values = file.column_values(lst_file_by_line)
    lst_couple = file.values_to_couple_PlanFact(lst_column_values)
    success_workers_lst = file.difference(lst_couple)
    workers = file.name_workers(sheet, max_col)
    success_workers_dict = file.success_dict(workers, success_workers_lst)

    return success_workers_dict



# совмещение данных из файлов в один словарь
def combination_dicts(lst_dicts):

    # содержит фио сотрудника и его успешность
    success_workers_dict_all_files = dict()

    # множество позволит не обрабатывать одних 
    # и тех же сотрудников несколько раз
    unic_workers = set()

    # перебор словарей
    for i in range(len(lst_dicts)):

        # key_i - рассматриваемый сотрудник
        for key_i in lst_dicts[i].keys():
            # values_lst - список успешностей для одного сотрудника из разных файлов
            values_lst = [lst_dicts[i].get(key_i)]

            # остальные словари
            for j in range(i+1, len(lst_dicts)):

                # если сотрудник упоминается в другом файле
                if key_i in lst_dicts[j].keys():
                    unic_workers.add(key_i)
                    values_lst.append(lst_dicts[j].get(key_i))
                # к итоговому словарю добавляем пару фио сотрудника - его успешность
                success_workers_dict_all_files[key_i] = sum(values_lst) 

            # если сотрудник есть только в одном файле,
            # его успешность добавляется без изменений
            if key_i not in unic_workers:
                success_workers_dict_all_files[key_i] = lst_dicts[i].get(key_i)
    
    return success_workers_dict_all_files



# обобщение на все файлы директории
def success_all_files():

    lst_dicts = list()
    # чтение всех файлов из директории и запись сотрудников и их
    # успешностей по отдельным словарям в список lst_dicts
    for elem in os.listdir():
        if elem[-4:] == 'xlsx':
            lst_dicts.append(success_by_one_file(elem))

    success_workers_dict_all_files = combination_dicts(lst_dicts)

    # печать сотрудников по успешности
    for elem in sorted(success_workers_dict_all_files, 
            key=success_workers_dict_all_files.get, reverse=True):
        print(elem)


try:
    success_all_files()
except ValueError:
    print('Некорректные данные в ячейках с человеко-часами!')
except PermissionError:
    print('Необходимо закрыть все файлы перед их обработкой!')

Иванов Р.А.
Сидоров М.В.
Петров И.И.
Сабитов А.В.
